# Reading events from RGTDB

Query to get JSON:

https://rgtdb.com/events/json?search=&offset=0&limit=100

This reads upcoming events from rgtdb.com and converts them into iCAL format and writes to a file for manual import. Then this saves or updates the events in a Google calendar.

## Get data and cache response

In [46]:
import pandas as pd 

import datetime
from datetime import datetime as dt

import io
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("Debug level logging turned on")

import requests
from cachecontrol import CacheControl
from cachecontrol.caches import FileCache
from cachecontrol.heuristics import ExpiresAfter

sess = requests.session()
cached_sess = CacheControl(sess, cache = FileCache('.web_cache'), heuristic=ExpiresAfter(hours=1))

try:
    response = cached_sess.get('https://rgtdb.com/events/json?search=&offset=0&limit=200') # Get 200 events. Should be about a week's worth of events
    response.raise_for_status()

except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

logger.setLevel(logging.ERROR)

print(dt.now())

DEBUG:root:Debug level logging turned on
DEBUG:cachecontrol.controller:Looking up "https://rgtdb.com/events/json?search=&offset=0&limit=200" in the cache
DEBUG:cachecontrol.controller:Current age based on date: 39141
DEBUG:cachecontrol.controller:Freshness lifetime from expires: 3601
DEBUG:cachecontrol.controller:The cached response is "stale" with no etag, purging
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): rgtdb.com:443


In [32]:
response.json().keys()

dict_keys(['total', 'rows'])

## Convert JSON to Pandas Dataframe

Not necessary, but hey, pretending to be a data scientist feels cool.

In [33]:
df = pd.json_normalize(response.json(), 'rows')
df

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
0,MRETT16 Siboire CCS,03-14 13:30,/events/76973,[race],3,25.37 km,96 m,94 m,MRETT16 - Mumbai,/courses/152577,False
1,Lou's Sunday Group Ride,03-14 15:00,/events/76622,[groupride],109,38.22 km,148 m,148 m,Kielder,/courses/47238,False
2,Sunday Social,03-14 15:00,/events/73727,[groupride],13,35.31 km,450 m,450 m,Dirty Reiver,/courses/80194,False
3,Sunday’s Ladies Ride,03-14 16:30,/events/78380,[groupride],5,12.21 km,1.44 km,9 m,Mt_Washington_hill_climb,/courses/155563,False
4,ToAD Downer Handcycles,03-14 16:40,/events/76566,[race],33,13.92 km,45 m,45 m,ToAD_Downer_Ave,/courses/150844,False
...,...,...,...,...,...,...,...,...,...,...,...
195,SDW Friday Frenzy #7,03-26 18:00,/events/78559,"[race, ranked]",13,25.31 km,218 m,218 m,SDW Cyclopark (Clockwise),/courses/155819,True
196,R2G Friday Night Crit,03-26 18:30,/events/70078,"[race, ranked]",3,22.07 km,106 m,106 m,Pdq Odd Down Circuit,/courses/107189,True
197,#100 Combe Gibbet,03-26 19:30,/events/62635,[groupride],2,5.50 km,118 m,176 m,#100 Combe Gibbet,/courses/110781,False
198,WKG's SDW. Stage 3,03-27 06:00,/events/61493,[race],1,26.96 km,635 m,469 m,SDW Amberley To Devi,/courses/107881,False


## Convert start time into datetime format - and guess at the year

This will be an issue every year around new year.

In [34]:

this_year = str(dt.today().year)
df['date'] = pd.to_datetime(df['startAt'] + ' ' + this_year, format='%m-%d %H:%M %Y', utc=True)
df.set_index('date', inplace=True)


In [35]:
df.head()

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
date,,,,,,,,,,,
2021-03-14 13:30:00+00:00,MRETT16 Siboire CCS,03-14 13:30,/events/76973,[race],3,25.37 km,96 m,94 m,MRETT16 - Mumbai,/courses/152577,False
2021-03-14 15:00:00+00:00,Lou's Sunday Group Ride,03-14 15:00,/events/76622,[groupride],109,38.22 km,148 m,148 m,Kielder,/courses/47238,False
2021-03-14 15:00:00+00:00,Sunday Social,03-14 15:00,/events/73727,[groupride],13,35.31 km,450 m,450 m,Dirty Reiver,/courses/80194,False
2021-03-14 16:30:00+00:00,Sunday’s Ladies Ride,03-14 16:30,/events/78380,[groupride],5,12.21 km,1.44 km,9 m,Mt_Washington_hill_climb,/courses/155563,False
2021-03-14 16:40:00+00:00,ToAD Downer Handcycles,03-14 16:40,/events/76566,[race],33,13.92 km,45 m,45 m,ToAD_Downer_Ave,/courses/150844,False


## Use icalendar package to create ICAL format events

* GitHub: https://github.com/collective/icalendar


In [36]:
from datetime import timedelta
from icalendar import vCalAddress, vText
from icalendar import Calendar, Event
import pytz

cal = Calendar()
cal.add('prodid', '-//My calendar product//mxm.com//')
cal.add('version', '2.0')

for index, row in df.iterrows():
    print(index, row['name'], row['tags'])
    event = Event()
    event['uid'] = row['detailsUrl']
    event.add('summary', str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']))
    event.add('dtstart', index)
    event.add('dtend', index  + timedelta(hours=1))
    event.add('url', 'https://rgtdb.com' + row['detailsUrl'])
    event.add('description', row['distance'] + ' ' + 'https://rgtdb.com' + row['detailsUrl'])
    event.add('color', 'Tomato')
    event['location'] = vText(row['roadName'])

    cal.add_component(event)

2021-03-14 13:30:00+00:00 MRETT16 Siboire CCS ['race']
2021-03-14 15:00:00+00:00 Lou's Sunday Group Ride ['groupride']
2021-03-14 15:00:00+00:00 Sunday Social ['groupride']
2021-03-14 16:30:00+00:00 Sunday’s Ladies Ride ['groupride']
2021-03-14 16:40:00+00:00 ToAD Downer Handcycles ['race']
2021-03-14 17:00:00+00:00 U15 | Race #1 ['race']
2021-03-14 17:00:00+00:00 BCG Polka Dot Series 3of3 ['groupride']
2021-03-14 17:20:00+00:00 ToAD Downer Pro Women ['race', 'pro']
2021-03-14 18:00:00+00:00 MRETT16 Pedalers ['race']
2021-03-14 18:00:00+00:00 OTR GREEN RACE MRETT16 ['race']
2021-03-14 18:00:00+00:00 Race to the Light House ['race']
2021-03-14 19:00:00+00:00 Weekend Warrior ['groupride']
2021-03-14 19:30:00+00:00 #100 Cheshire ['groupride']
2021-03-14 20:00:00+00:00 Sunday Social ['groupride']
2021-03-14 22:00:00+00:00 Flat Out Flyer ['race']
2021-03-15 00:00:00+00:00 Tempo Tester ['groupride']
2021-03-15 01:00:00+00:00 Gran Premio Pienza ['race']
2021-03-15 01:00:00+00:00 Flat Out Flye

## Write to File

Can use to manually import into Google, other calendars

In [37]:
import tempfile, os
f = open('./rgt_events.ics', 'wb')
f.write(cal.to_ical())
f.close()

## Use gcsa for Simplified Access to Google Calendar API

* GitHub: https://github.com/kuzmoyev/google-calendar-simple-api
* Docs: https://google-calendar-simple-api.readthedocs.io/en/latest/index.html

Need to add socket timeout of 5 minutes due to slow response on my machine

### Shared Calendar

* Calendar ID: 3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com
* Google sharing link: https://calendar.google.com/calendar/u/0?cid=M2U4Z2F1OGJvbW1mamszM2o5MnJ2NWs3cTBAZ3JvdXAuY2FsZW5kYXIuZ29vZ2xlLmNvbQ
* ICS format for e.g., Apple Calendar: https://calendar.google.com/calendar/ical/3e8gau8bommfjk33j92rv5k7q0%40group.calendar.google.com/public/basic.ics


In [38]:
from gcsa.event import Event as gcEvent
from gcsa.google_calendar import GoogleCalendar
from gcsa.recurrence import Recurrence, DAILY, SU, SA

import socket
socket.setdefaulttimeout(300) # 5 minutes

EMAIL_FOR_CAL = '3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com'

calendar = GoogleCalendar(EMAIL_FOR_CAL)


## Cleanup: Delete All Events from Google Calendar

Uncomment to clean up calendar.

In [39]:
#calendar.clear() # This gives an error in the Google API

#for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):
#    print('Deleting:', event, event.event_id)
#    calendar.delete_event(event)

## Find existing events, mark for update instead of creation

In [40]:
import re

reExtractName = re.compile(" \[\'.*")

df['cal_id'] = None
df['event_obj'] = None
df['found'] = False

for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):

    print(event)

    rideName = reExtractName.sub("", event.summary) # Get substring from event summary with just the name

    df.loc[(df['name'] == rideName) & (df.index == event.start), ['found', 'cal_id', 'event_obj']] = [True, event.event_id, event]


2021-03-13 19:30:00+00:00 - #100 Lancashire ['groupride'] 1/31.63 km/673 m
2021-03-14 12:30:00+00:00 - #100 Fford Penlech ['groupride'] 1/6.04 km/192 m
2021-03-14 17:00:00+00:00 - BCG Polka Dot Series 3of3 ['groupride'] 3/32.52 km/1.42 km
2021-03-14 19:30:00+00:00 - #100 Cheshire ['groupride'] 1/25.49 km/596 m
2021-03-15 12:30:00+00:00 - #100 Bwlch y Groes ['groupride'] 1/6.16 km/394 m
2021-03-15 19:30:00+00:00 - #100 Devils Staircase ['groupride'] 1/4.46 km/179 m
2021-03-16 12:30:00+00:00 - #100 Aberdare ['groupride'] 1/44.32 km/1.15 km
2021-03-16 19:30:00+00:00 - #100 Swansea ['groupride'] 1/44.61 km/713 m
2021-03-17 12:30:00+00:00 - #100 North Wales 1 ['groupride'] 1/44.87 km/1.11 km
2021-03-17 19:30:00+00:00 - #100 North Wales 2 ['groupride'] 1/36.20 km/1.05 km
2021-03-18 12:30:00+00:00 - #100 The Wall ['groupride'] 1/16.98 km/242 m
2021-03-18 19:30:00+00:00 - #100 Lake District 1 ['groupride'] 1/41.59 km/1.51 km
2021-03-19 12:30:00+00:00 - #100 Lake District 2 ['groupride'] 1/39.8

In [41]:
df.loc[df['found'] == True]


,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-03-14 13:30:00+00:00,MRETT16 Siboire CCS,03-14 13:30,/events/76973,[race],3,25.37 km,96 m,94 m,MRETT16 - Mumbai,/courses/152577,False,29nmmdaf6vn8qmlceqo0shrjh8,2021-03-14 13:30:00+00:00 - MRETT16 Siboire CC...,True
2021-03-14 15:00:00+00:00,Lou's Sunday Group Ride,03-14 15:00,/events/76622,[groupride],109,38.22 km,148 m,148 m,Kielder,/courses/47238,False,p8s6d66369cvgnk5oapfbkec7c,2021-03-14 15:00:00+00:00 - Lou's Sunday Group...,True
2021-03-14 15:00:00+00:00,Sunday Social,03-14 15:00,/events/73727,[groupride],13,35.31 km,450 m,450 m,Dirty Reiver,/courses/80194,False,l154jej4fjapknqnainla1q3t0,2021-03-14 15:00:00+00:00 - Sunday Social ['gr...,True
2021-03-14 16:30:00+00:00,Sunday’s Ladies Ride,03-14 16:30,/events/78380,[groupride],5,12.21 km,1.44 km,9 m,Mt_Washington_hill_climb,/courses/155563,False,eurf5mmbk4vch08u77nbd3fco0,2021-03-14 16:30:00+00:00 - Sunday’s Ladies Ri...,True
2021-03-14 16:40:00+00:00,ToAD Downer Handcycles,03-14 16:40,/events/76566,[race],33,13.92 km,45 m,45 m,ToAD_Downer_Ave,/courses/150844,False,d3rnger5ta8ll923l5pvg4fkrc,2021-03-14 16:40:00+00:00 - ToAD Downer Handcy...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-25 19:30:00+00:00,#100 Five Peaks,03-25 19:30,/events/62608,[groupride],1,38.28 km,948 m,881 m,#100 5Peak Challenge,/courses/110793,False,udkrdrfmeuosh02k4rdb71h66s,2021-03-25 19:30:00+00:00 - #100 Five Peaks ['...,True
2021-03-26 12:30:00+00:00,#100 Salcombe Hill,03-26 12:30,/events/62634,[groupride],1,5.68 km,198 m,84 m,#100 Salcombe Hill,/courses/110785,False,k0sgntn8ctdaju92agqte7allg,2021-03-26 12:30:00+00:00 - #100 Salcombe Hill...,True
2021-03-26 18:30:00+00:00,R2G Friday Night Crit,03-26 18:30,/events/70078,"[race, ranked]",3,22.07 km,106 m,106 m,Pdq Odd Down Circuit,/courses/107189,True,irrd1e5clkmko90u24g7ifmjqk,2021-03-26 18:30:00+00:00 - R2G Friday Night C...,True


In [42]:
df.loc[df['found'] == False]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-03-14 18:00:00+00:00,OTR GREEN RACE MRETT16,03-14 18:00,/events/78951,[race],6,25.37 km,96 m,94 m,MRETT16 - Mumbai,/courses/156895,False,None,None,False
2021-03-17 21:00:00+00:00,Greenwich Splunk WNR,03-17 21:00,/events/79128,[groupride],3,59.24 km,724 m,668 m,Greenwich WNR Steps,/courses/97312,False,None,None,False
2021-03-20 14:00:00+00:00,OTR March Monument Race 3,03-20 14:00,/events/79098,"[race, ranked]",2,96.36 km,320 m,327 m,ParisRoubaix 3,/courses/157205,True,None,None,False
2021-03-24 18:00:00+00:00,Mountain Goats,03-24 18:00,/events/76997,"[race, ranked]",6,10.12 km,840 m,3 m,Pla D'Adet,/courses/152397,True,None,None,False
2021-03-25 10:50:00+00:00,KISS At Base,03-25 10:50,/events/72528,[groupride],1,70.63 km,900 m,900 m,Dirty Reiver,/courses/80194,False,None,None,False
2021-03-25 17:45:00+00:00,La classique du Jeudi !,03-25 17:45,/events/73306,"[race, ranked]",9,15.27 km,53 m,48 m,Marie Galante,/courses/145221,True,None,None,False
2021-03-25 18:25:00+00:00,La classique du Jeudi !,03-25 18:25,/events/73309,"[race, itt, ranked]",10,5.16 km,538 m,0 m,Basse-Terre Matuba,/courses/145485,True,None,None,False
2021-03-26 18:00:00+00:00,SDW Friday Frenzy #7,03-26 18:00,/events/78559,"[race, ranked]",13,25.31 km,218 m,218 m,SDW Cyclopark (Clockwise),/courses/155819,True,None,None,False
2021-03-27 08:00:00+00:00,WESTERLEY CC OPEN 10,03-27 08:00,/events/73962,"[race, itt, ranked]",23,16.11 km,70 m,77 m,Gt.Missenden H4-10 R,/courses/27081,True,None,None,False


## Add All Events to Google Calendar

In [43]:
from gcsa.event import Event as gcEvent

for index, row in df.iterrows():

    evntSummary = str(str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']) + '/' + row['distance'] + '/' + row['elevationGain'])

    evntDescription = 'Signups: ' +  str(row['signUps']) + '\n' + 'Distance: ' + row['distance'] + '\n' +  'Elevation gain: ' + row['elevationGain'] + '\n' + 'Descent: ' + row['elevationLost'] + '\n' + 'https://rgtdb.com' + str(row['detailsUrl'])

    evntString = row['detailsUrl'].replace('/events/', '')

    if row['found'] == False:

        print('+New event: ', index, row['name'], row['tags'], evntString)

        evntColor = '1'

        if "groupride" in row['tags']:
            evntColor = '2'
        elif 'pro' in row['tags']:
            evntColor = '3'
        elif 'elimination' in row['tags']:
            evntColor = '4'
        elif "itt" in row['tags']:
            evntColor = '5'
        elif "race" in row['tags']:
            evntColor = '6'

        event = gcEvent(
            evntSummary,
            start=index,
            timezone='UTC',
            location=str(row['roadName']),
            description=evntDescription,
            event_id=evntString,
            color = evntColor
        )

        print('ID before add:', event.event_id)
        ret_event = calendar.add_event(event)
        print('ID after add:', event.event_id, 'Returned event ID:', ret_event.event_id)

    else:

        print('-Updating event: ', index, row['name'], row['tags'], evntString)

        event = row['event_obj']

        event.summary = evntSummary
        event.description = evntDescription

        calendar.update_event(event)

-Updating event:  2021-03-14 13:30:00+00:00 MRETT16 Siboire CCS ['race'] 76973
-Updating event:  2021-03-14 15:00:00+00:00 Lou's Sunday Group Ride ['groupride'] 76622
-Updating event:  2021-03-14 15:00:00+00:00 Sunday Social ['groupride'] 73727
-Updating event:  2021-03-14 16:30:00+00:00 Sunday’s Ladies Ride ['groupride'] 78380
-Updating event:  2021-03-14 16:40:00+00:00 ToAD Downer Handcycles ['race'] 76566
-Updating event:  2021-03-14 17:00:00+00:00 U15 | Race #1 ['race'] 77065
-Updating event:  2021-03-14 17:00:00+00:00 BCG Polka Dot Series 3of3 ['groupride'] 72387
-Updating event:  2021-03-14 17:20:00+00:00 ToAD Downer Pro Women ['race', 'pro'] 76569
-Updating event:  2021-03-14 18:00:00+00:00 MRETT16 Pedalers ['race'] 77846
+New event:  2021-03-14 18:00:00+00:00 OTR GREEN RACE MRETT16 ['race'] 78951
ID before add: 78951
ID after add: 78951 Returned event ID: 9ui571i2sve8rgtt75metae5u8
-Updating event:  2021-03-14 18:00:00+00:00 Race to the Light House ['race'] 73470
-Updating even

In [44]:
df.loc[df['signUps'] > 9]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-03-14 15:00:00+00:00,Lou's Sunday Group Ride,03-14 15:00,/events/76622,[groupride],109,38.22 km,148 m,148 m,Kielder,/courses/47238,False,p8s6d66369cvgnk5oapfbkec7c,2021-03-14 15:00:00+00:00 - Lou's Sunday Group...,True
2021-03-14 15:00:00+00:00,Sunday Social,03-14 15:00,/events/73727,[groupride],13,35.31 km,450 m,450 m,Dirty Reiver,/courses/80194,False,l154jej4fjapknqnainla1q3t0,2021-03-14 15:00:00+00:00 - Sunday Social ['gr...,True
2021-03-14 16:40:00+00:00,ToAD Downer Handcycles,03-14 16:40,/events/76566,[race],33,13.92 km,45 m,45 m,ToAD_Downer_Ave,/courses/150844,False,d3rnger5ta8ll923l5pvg4fkrc,2021-03-14 16:40:00+00:00 - ToAD Downer Handcy...,True
2021-03-14 17:00:00+00:00,U15 | Race #1,03-14 17:00,/events/77065,[race],61,16.05 km,187 m,187 m,Tiegem Challenge,/courses/149868,False,249n7peqjdg5pqsp6mvgjf752g,2021-03-14 17:00:00+00:00 - U15 | Race #1 ['ra...,True
2021-03-14 17:20:00+00:00,ToAD Downer Pro Women,03-14 17:20,/events/76569,"[race, pro]",21,48.72 km,158 m,159 m,ToAD_Downer_Ave,/courses/150844,False,lrsip8nh18ttgbh5bbf77hh92k,2021-03-14 17:20:00+00:00 - ToAD Downer Pro Wo...,True
2021-03-14 18:00:00+00:00,Race to the Light House,03-14 18:00,/events/73470,[race],11,22.43 km,515 m,367 m,Cap Formentor,/courses/106,False,j5k23285lg4bviv6ebkp110k3k,2021-03-14 18:00:00+00:00 - Race to the Light ...,True
2021-03-15 19:00:00+00:00,CTT evening 10,03-15 19:00,/events/76984,"[race, itt, ranked]",17,16.47 km,48 m,61 m,CTT V718 (Virtual),/courses/22953,True,t26sogdmbq8rt96rilkudkhl94,2021-03-15 19:00:00+00:00 - CTT evening 10 ['r...,True
2021-03-16 12:30:00+00:00,Herd Take it Easy Tuesday,03-16 12:30,/events/69722,[groupride],12,24.41 km,17 m,12 m,Sunk Island To Spurn,/courses/98661,False,jopsca6pibc39uojf6e5jb7p18,2021-03-16 12:30:00+00:00 - Herd Take it Easy ...,True
2021-03-16 17:30:00+00:00,Tour Series : race 3 (b),03-16 17:30,/events/68044,"[race, ranked]",10,28.28 km,195 m,195 m,Aberystwyth Circuit,/courses/108729,True,g5t7d4inba5phkmgv96nfj1h08,2021-03-16 17:30:00+00:00 - Tour Series : race...,True


In [45]:
df.loc[df['name'].str.startswith('Wacky Races')]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
